In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%pwd

'/content'

In [3]:
%cd drive/MyDrive/milestone_3/notebooks/rl_training

/content/drive/MyDrive/milestone_3/notebooks/rl_training


In [4]:
!pip install torch==1.13.1
!pip install -U bitsandbytes
!pip install -U git+https://github.com/huggingface/transformers.git 
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets
!pip install trl
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-m_q077z5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-m_q077z5
  Resolved https://github.com/huggingface/transformers.git to commit 8f093fb799246f7dd9104ff44728da0c53a9f67a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-85luawm5
  Running command git c

In [1]:
import sys
MODULE_PATHS = ["../../modules"]
for module_path in MODULE_PATHS:
    if module_path not in sys.path:
        sys.path.append(module_path)

In [2]:
import json

import torch
torch.manual_seed(42)

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from trl import AutoModelForSeq2SeqLMWithValueHead
# from peft import TaskType, LoraConfig, LoraConfig, get_peft_model

from ppo_training import PPOTrainerWrapper, build_ppo_hf_dataset
from rewards import RewardModel

%load_ext autoreload
%autoreload 2


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/farouk/anaconda3/envs/mnlp-py3-9/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/farouk/anaconda3/envs/mnlp-py3-9/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/home/farouk/anaconda3/envs/mnlp-py3-9/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/farouk/anaconda3/envs/mnlp-py3-9/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/farouk/anaconda3/envs/mnlp-py3-9 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/farouk/anaconda3/envs/mnlp-py3-9/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/share/gconf/cinnamon.default.path')}
  warn(msg)
/home/farouk/anaconda3/envs/mnlp-py3-9/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: U

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/farouk/anaconda3/envs/mnlp-py3-9/bin/python: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 00
CUDA SETUP: Loading binary /home/farouk/anaconda3/envs/mnlp-py3-9/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


### Models

In [3]:
device = "cuda"

In [8]:
# REWARD MODEL
reward_model = RewardModel.from_pretrained("../../models/reward-model").to(device)

# PEFT MODEL
peft_model_path = "../../models/assistant-ppo" # continue training
config = PeftConfig.from_pretrained(peft_model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
model = PeftModel.from_pretrained(model, peft_model_path, is_trainable=True)

# TOKENIZER FOR THE PEFT MODEL
tokenizer_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)

model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(
    model, device_map="auto"
)

In [9]:
model.pretrained_model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096414524241463


In [10]:
sum([p.numel() for p in model.parameters() if p.requires_grad])

1770241

### Dataset

In [11]:
with open("../../submission_items/gen_dataset_NotAnAGI.json", "r") as reader:
    dataset, _ = build_ppo_hf_dataset(data=json.load(reader), tokenizer=tokenizer, gen_margin=100)
    dataset = dataset.shard(num_shards=5, index=1) # continue training with next shard
    print(len(dataset))

Map:   0%|          | 0/10647 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10647 [00:00<?, ? examples/s]

6746


### Training

In [13]:
ppo_config = PPOTrainerWrapper.create_ppo_config(
    model_name="flan-t5-base-assistant-ppo",
    batch_size=8,
    mini_batch_size=1,
    grad_cumul_steps=8
)

gen_config = {
        "num_return_sequences": 1,
        "min_length": 16,
        "max_length": tokenizer.model_max_length,
        
        "do_sample": True,
        "top_k": 0,
        "top_p": 0.4,
        "repetition_penalty": 2,
        "no_repeat_ngram_size": 4,
        
        "use_cache": True,
        "pad_token_id": tokenizer.pad_token_id
}

trainer = PPOTrainerWrapper(
    config=ppo_config,
    model=model, tokenizer=tokenizer, reward_model=reward_model, gen_config=gen_config,
    dataset=dataset,
    optim_name="lion"
)

number of optimizable parameters : 1770241
model device: cuda:0
reward device: cuda:0


In [16]:
run_id = "run_06131200"

# training
save_path = f"../../models/assistant-ppo-training/{run_id}"
trainer.train(save_path=save_path, save_freq=12)

100%|██████████| 168/168 [2:54:54<00:00, 62.47s/it]


In [17]:
run_id = "run_06131200"
epoch  = "last"
model_path = f"../../models/assistant-ppo-training/{run_id}/{epoch}"

config = PeftConfig.from_pretrained(model_path)
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
loaded_model = PeftModel.from_pretrained(loaded_model, peft_model_path, is_trainable=True)

In [45]:
tokenized = tokenizer(["Question: What is principle component analysis? Answer:"],
                            padding="max_length", truncation=True, max_length=tokenizer.model_max_length,
                            return_tensors="pt")
tokenizer.decode(
    loaded_model.generate(**tokenized, min_length=50, max_length=50, do_sample=True, top_p=0.25, top_k=0).squeeze().tolist(),
    skip_special_tokens=True)

'The principle component analysis is a mathematical analysis of the structure of a component in a given set of variables. It is used to evaluate the properties of a component in a given set of variables. The first component is the element'